In [1]:
import torch as th
import timeit


from simplesim.risk import Risk
from simplesim.chain import Chain
from simplesim.module import Module
from simplesim.product import Product
from simplesim.csv_input import CSVInput

from simplesim.simulation.simulation import Simulation
from simplesim.simulation.risk_generator import RiskGen

In [2]:
device = 'cuda' if th.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# Input

data_dir = CSVInput('data')
risk_assembly = data_dir.get_risk_assembly()

riskgen = RiskGen(risk_assembly, parallel_runs=1, batch_size=1)
riskgen.get_risk_tensor()


  name                             description  \
0    A                         Data is missing   
1    B                   Different data format   
2    C                   Cyber-security issues   
3    D           Issues with machines/furnaces   
4    E  Deviations in material characteristics   
5    F                  Dependency on supplier   
6    G                       Machine breakdown   

                                          disruption        modules  \
0                                      Order is late  sss, ssa, ssb   
1                              Production is blocked       dde, sss   
2      Critical data is not shared with stakeholders       ddr, sss   
3                              Production is blocked       ddt, sss   
4             Delay in supplying product to customer       ffg, sss   
5  Lack of redundancy leads to disruption in oper...       ffb, sss   
6                                                NaN       ffb, sss   

  probability capacity reducti

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.